<a href="https://colab.research.google.com/github/grzegorzkwolek/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/GKwolek_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [X] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html). Use the scaler's `fit_transform` method with the train set. Use the scaler's `transform` method with the test set.
- [X] Fit a ridge regression model with multiple features.
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import numpy as np
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
df.sample(10)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
6102,1,UPPER EAST SIDE (59-79),10 COOPS - ELEVATOR APARTMENTS,2,1428,17,NaN,D4,"235 EAST 73RD STREET, 4C",NaN,10021.0,0.0,0.0,0.0,0,0.0,1936.0,2,D4,1252000,01/31/2019
3543,4,OTHER,01 ONE FAMILY DWELLINGS,1,12461,5,NaN,A1,17827 119 RD,NaN,11434.0,1.0,0.0,1.0,"2,500",1812.0,1925.0,1,A1,335000,01/18/2019
18308,3,OTHER,03 THREE FAMILY DWELLINGS,1,7356,70,NaN,C0,2113 EAST 23RD STREET,NaN,11229.0,3.0,0.0,3.0,"2,500",3924.0,1930.0,1,C0,0,04/01/2019
4852,3,OTHER,02 TWO FAMILY DWELLINGS,1,4111,107,NaN,B2,403 ETNA STREET,NaN,11208.0,2.0,0.0,2.0,"1,488",1488.0,1899.0,1,B2,0,01/25/2019
6902,4,ASTORIA,01 ONE FAMILY DWELLINGS,1,1023,14,NaN,A5,28-18 56TH PLACE,NaN,11377.0,1.0,0.0,1.0,"2,167",1697.0,1940.0,1,A5,708000,02/04/2019
16635,3,BOROUGH PARK,03 THREE FAMILY DWELLINGS,1,5630,61,NaN,C0,1441 48TH STREET,NaN,11219.0,3.0,0.0,3.0,"4,007",3600.0,1920.0,1,C0,10,03/25/2019
5348,5,OTHER,01 ONE FAMILY DWELLINGS,1,6869,63,NaN,A0,1686 CARLTON AVENUE,NaN,10309.0,1.0,0.0,1.0,"3,800",1182.0,1955.0,1,A0,0,01/28/2019
11826,3,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,6779,4,NaN,D4,"1610 AVENUE P, 5C",NaN,11229.0,0.0,0.0,0.0,0,0.0,1933.0,2,D4,0,02/28/2019
15260,3,OTHER,01 ONE FAMILY DWELLINGS,1,5191,73,NaN,A5,333 EAST 25TH STREET,NaN,11226.0,1.0,0.0,1.0,"2,000",1776.0,1899.0,1,A5,0,03/18/2019
10361,4,FLUSHING-NORTH,10 COOPS - ELEVATOR APARTMENTS,2,4342,10,NaN,D4,"138-12 28TH RD, 4E",NaN,11354.0,0.0,0.0,0.0,0,0.0,1952.0,2,D4,360000,02/21/2019


In [0]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [0]:
useless = ['EASE-MENT', 'APARTMENT_NUMBER']
df.drop(useless, axis=1, inplace=True)

In [0]:
df = df.dropna()

In [0]:
#changing types
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
df['ZIP_CODE'] = df['ZIP_CODE'].astype(int)
df['YEAR_BUILT'] = df['YEAR_BUILT'].astype(int)

In [0]:
#an hour wasted to understand this "########"
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(r',', '')
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace('########', '')

In [0]:
df = df.dropna()

In [0]:
df['LAND_SQUARE_FEET'] = pd.to_numeric(df['LAND_SQUARE_FEET'])
df.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
ZIP_CODE                                   int64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                         float64
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                                 int64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE                         datetime64[ns]
dtype: object

In [0]:
df.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  1
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [0]:
df = df.dropna()

In [0]:
df.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [0]:
#mask 1
class_mask = (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')
subset_df = df[class_mask]

In [0]:
#mask 2
price_mask = (subset_df['SALE_PRICE'] > 100000) & (subset_df['SALE_PRICE'] < 2000000)
subset_df = subset_df[price_mask]

In [0]:
subset_df['SALE_DATE'] = pd.to_datetime(subset_df['SALE_DATE'])

In [0]:
subset_df['SALE_DATE'].describe()

count                    3151
unique                     91
top       2019-01-31 00:00:00
freq                       78
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [0]:
#remember, no leading zeros in pd.datetime
cutoff_date = pd.datetime(2019,4,1)
train_mask = (subset_df['SALE_DATE'] < cutoff_date)
test_mask = (subset_df['SALE_DATE'] >= cutoff_date)

In [0]:
train_df = subset_df[train_mask]
test_df = subset_df[test_mask]

In [0]:
train_df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230,1.0,0.0,1.0,6800.0,1325.0,1930,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427,1.0,0.0,1.0,4000.0,2001.0,1940,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461,1.0,0.0,1.0,3500.0,2043.0,1925,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226,1.0,0.0,1.0,4000.0,2680.0,1899,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203,1.0,0.0,1.0,1710.0,1872.0,1940,1,A5,620000,2019-01-02


In [0]:
train_df.select_dtypes(exclude='number').describe()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,2507,2507
unique,5,6,1,2,13,2497,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,104-18 187TH STREET,A1,2019-01-31 00:00:00
freq,1204,2382,2507,2476,919,2,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [0]:
target = 'SALE_PRICE'
high_card = ['ADDRESS', 'BUILDING_CLASS_CATEGORY', 'SALE_DATE']
features = train_df.columns.drop(high_card + [target])

In [0]:
features

Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [0]:
X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]

In [0]:
import category_encoders as ce 
encoder = ce.OneHotEncoder(use_cat_names = True)

In [0]:
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [0]:
X_train.shape

(2507, 47)

In [0]:
X_test.shape

(644, 47)

In [0]:
from sklearn.feature_selection import f_regression, SelectKBest
selector = SelectKBest(score_func=f_regression, k=25)

In [0]:
X_train_select = selector.fit_transform(X_train, y_train);
X_test_select = selector.transform(X_test);

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [0]:
X_train_select.shape

(2507, 25)

In [0]:
X_test_select.shape

(644, 25)

In [0]:
column_names = X_train.columns
select_mask = selector.get_support()
select_names = column_names[select_mask]
unselect_names = column_names[~select_mask]

print('Selected Features ')
for name in select_names:
  print(name)

print('')
print('Ignored Selected ')
for name in unselect_names:
  print(name)

Selected Features 
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_S0
ZIP_CODE
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_S0

Ignored Selected 
BOROUGH_1
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_B3
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE
BUILDING_CLASS_AT_TIME_OF_SA

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [0]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [0]:
X_train_scaled

array([[ 2.3019546 , -0.96126036, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343, -0.96126036,  3.05933067, ..., -0.07493813,
        -0.11189367, -0.01997604],
       ...,
       [-0.43441343, -0.96126036, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343, -0.96126036, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604]])

In [0]:
X_test_scaled

array([[-0.43441343, -0.96126036,  3.05933067, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343, -0.96126036,  3.05933067, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [ 2.3019546 , -0.96126036, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       ...,
       [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604],
       [-0.43441343,  1.04030088, -0.32686888, ..., -0.07493813,
        -0.11189367, -0.01997604]])

In [0]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=100)

In [0]:
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [0]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, y_pred)
display((f'Mean Absolute Error on the Test sample is ${test_mae:,.2f}'))

'Mean Absolute Error on the Test sample is $155,327.12'

In [0]:
base_mad = df['SALE_PRICE'].mad()
base_mean = df['SALE_PRICE'].mean()
display((f'Mean Absolute Deviation on the Base is ${base_mad:,.2f} vs. the mean of ${base_mean:,.2f}'))

'Mean Absolute Deviation on the Base is $1,616,843.69 vs. the mean of $1,327,321.98'

In [0]:
df['SALE_PRICE'].describe()

count    2.295200e+04
mean     1.327322e+06
std      1.027261e+07
min      0.000000e+00
25%      0.000000e+00
50%      4.512300e+05
75%      8.750000e+05
max      8.500000e+08
Name: SALE_PRICE, dtype: float64